In [1]:
import simplejson as json
import time
import smart_open
import re
import sys
import os

In [7]:
#!python -m gensim.scripts.segment_wiki -i -f enwiki-20200101-pages-articles-multistream1.xml-p10p30302.bz2 -o enwiki-latest.json.gz

In [2]:
RE_P0 = re.compile('<!--.*?-->', re.DOTALL | re.UNICODE)  # comments
RE_P1 = re.compile('<ref([> ].*?)(</ref>|/>)', re.DOTALL | re.UNICODE)  # footnotes
RE_P2 = re.compile("(\n\[\[[a-z][a-z][\w-]*:[^:\]]+\]\])+$", re.UNICODE)  # links to languages
RE_P3 = re.compile("{{([^}{]*)}}", re.DOTALL | re.UNICODE)  # template
RE_P4 = re.compile("{{([^}]*)}}", re.DOTALL | re.UNICODE)  # template
RE_P5 = re.compile('\[(\w+):\/\/(.*?)(( (.*?))|())\]', re.UNICODE)  # remove URL, keep description
RE_P6 = re.compile("\[([^][]*)\|([^][]*)\]", re.DOTALL | re.UNICODE)  # simplify links, keep description
RE_P7 = re.compile('\n\[\[[iI]mage(.*?)(\|.*?)*\|(.*?)\]\]', re.UNICODE)  # keep description of images
RE_P8 = re.compile('\n\[\[[fF]ile(.*?)(\|.*?)*\|(.*?)\]\]', re.UNICODE)  # keep description of files
RE_P9 = re.compile('<nowiki([> ].*?)(</nowiki>|/>)', re.DOTALL | re.UNICODE)  # outside links
RE_P10 = re.compile('<math([> ].*?)(</math>|/>)', re.DOTALL | re.UNICODE)  # math content
RE_P11 = re.compile('<(.*?)>', re.DOTALL | re.UNICODE)  # all other tags
RE_P12 = re.compile('\n(({\|)|(\|-)|(\|}))(.*?)(?=\n)', re.UNICODE)  # table formatting
RE_P13 = re.compile('\n(\||\!)(.*?\|)*([^|]*?)', re.UNICODE)  # table cell formatting
RE_P14 = re.compile('\[\[Category:[^][]*\]\]', re.UNICODE)  # categories
# Remove File and Image template
RE_P15 = re.compile('\[\[([fF]ile:|[iI]mage)[^]]*(\]\])', re.UNICODE)
RE_P16 = re.compile('=*\**', re.UNICODE)

def remove_markup(text):
    text = re.sub(RE_P2, "", text)  # remove the last list (=languages)
    iters = 0
    while True:
        old, iters = text, iters + 1
        text = re.sub(RE_P0, "", text)  # remove comments
        text = re.sub(RE_P1, '', text)  # remove footnotes
        text = re.sub(RE_P9, "", text)  # remove outside links
        text = re.sub(RE_P10, "", text)  # remove math content
        text = re.sub(RE_P11, "", text)  # remove all remaining tags
        text = re.sub(RE_P14, '', text)  # remove categories
        text = re.sub(RE_P5, '\\3', text)  # remove urls, keep description
        text = re.sub(RE_P6, '\\2', text)  # simplify links, keep description only
        # remove table markup
        text = text.replace('||', '\n|')  # each table cell on a separate line
        text = re.sub(RE_P12, '\n', text)  # remove formatting lines
        text = re.sub(RE_P13, '\n\\3', text)  # leave only cell content
        text = re.sub(RE_P16, '', text)  # leave only cell content
        # remove empty mark-up
        text = text.replace('[]', '')
        if old == text or iters > 2:  # stop if nothing changed between two iterations or after a fixed number of iterations
            break
    # the following is needed to make the tokenizer see '[[socialist]]s' as a single word 'socialists'
    text = text.replace('[', '').replace(']', '')  # promote all remaining markup to plain text
    return text

In [7]:
def save_article(title, text,  directory="./wikipedia/"):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            print(e)
        path=os.path.join(directory, title.replace(" ","_").replace("/","_") +'.txt')
        if os.path.exists(path):
            print('article already present')
        else:
            with open(path, 'w') as fp:
                fp.write(text)
    else:
        path=os.path.join(directory, title.replace(" ","_").replace("/","_") +'.txt')
        if os.path.exists(path):
            print('article already present')
        else:
            with open(path, 'w') as fp:
                fp.write(text)
    return None

In [9]:
if __name__== "__main__":
    filename="./enwiki-latest.json.gz"
    start_time=time.time()
    count=0
    try:
        for line in smart_open.open(filename):
            count+=1
            article = json.loads(line)
            title = article['title']
            print(title)
            text = remove_markup(" ".join(article['section_texts']))
            save_article(title, text)
    except Exception as e:
        print(e)
        print("In Exception")
        pass
    print("Total articles: %d" %count)
    print("total time(in min): %f"  %((time.time()-start_time)/60))

Anarchism
article already present
Autism
article already present
Albedo
article already present
A
article already present
Alabama
article already present
Achilles
article already present
Abraham Lincoln
article already present
Aristotle
article already present
An American in Paris
article already present
Academy Award for Best Production Design
article already present
Academy Awards
article already present
Actrius
article already present
Animalia (book)
article already present
International Atomic Time
article already present
Altruism
article already present
Ayn Rand
article already present
Alain Connes
article already present
Allan Dwan
article already present
Algeria
article already present
List of Atlas Shrugged characters
article already present
Anthropology
article already present
Agricultural science
article already present
Alchemy
article already present
Alien
article already present
Astronomer
article already present
ASCII
article already present
Austin (disambiguation)
article

article already present
Austrian School
article already present
Abscess
article already present
Aalborg Municipality
article already present
Aarhus
article already present
Northern cavefish
article already present
Abatement
article already present
Amateur
article already present
Alexis Carrel
article already present
All Souls' Day
article already present
Anatole France
article already present
André Gide
article already present
Algorithms for calculating variance
article already present
Almond
article already present
Demographics of Antigua and Barbuda
article already present
Politics of Antigua and Barbuda
article already present
Telecommunications in Antigua and Barbuda
article already present
Royal Antigua and Barbuda Defence Force
article already present
Antisemitism
article already present
Economy of Azerbaijan
article already present
Geography of Azerbaijan
article already present
Foreign relations of Azerbaijan
article already present
Azerbaijani Armed Forces
article already pres

article already present
August 19
article already present
August 21
article already present
Dodo (Alice's Adventures in Wonderland)
article already present
Lory (disambiguation)
article already present
Albert
article already present
Albert I
article already present
Albert II
article already present
Albert III
article already present
Albert Alcibiades, Margrave of Brandenburg-Kulmbach
article already present
Albert the Bear
article already present
Albert of Brandenburg
article already present
Albert, Duke of Prussia
article already present
August 25
article already present
Aachen
article already present
Agate
article already present
Aspirin
article already present
Abner
article already present
Ahmed I
article already present
Ahmed II
article already present
Ahmed III
article already present
Ainu people
article already present
Acropolis
article already present
Acupuncture
article already present
Adder
article already present
Aeneas
article already present
April 13
article already present

article already present
Allomorph
article already present
Allophone
article already present
Affix
article already present
Allegory
article already present
Allotropy
article already present
Agathocles of Syracuse
article already present
Economy of Alberta
article already present
Augustin-Louis Cauchy
article already present
Archimedes
article already present
Alternative medicine
article already present
Archimedean solid
article already present
Antiprism
article already present
Natural history of Africa
article already present
Geography of Africa
article already present
Approval voting
article already present
Arizona State University
article already present
April 14
article already present
Astoria, Oregon
article already present
Alarums and Excursions
article already present
Alfred Jarry
article already present
Amalric
article already present
Amalric of Jerusalem
article already present
Aimery of Cyprus
article already present
Anthemius of Tralles
article already present
Absalon
article 

article already present
Abdulaziz al-Omari
article already present
Aage Bohr
article already present
Analytic geometry
article already present
Arabic alphabet
article already present
Angels in art
article already present
Arctic fox
article already present
Anglo-Saxon (disambiguation)
article already present
Folklore of the United States
article already present
Sid Meier's Alpha Centauri
article already present
Abu Sayyaf
article already present
Armenian language
article already present
Additive synthesis
article already present
Aircraft carrier
article already present
Apicomplexa
article already present
Argentine cuisine
article already present
April 8
article already present
Ad hominem
article already present
Analysis of algorithms
article already present
Ælle of Sussex
article already present
Atari
article already present
Afghan
article already present
Acadia University
article already present
Steel-string acoustic guitar
article already present
Antipope John XXIII
article already pr

...

article already present
Sign
article already present
Standard Alphabet by Lepsius
article already present
Sidehill gouger
article already present
Strike
article already present
Second Vatican Council
article already present
Slovene language
article already present
Slovak language
article already present
Supercluster
article already present
Salicylic acid
article already present
Seaborgium
article already present
September 15
article already present
September 18
article already present
September 19
article already present
September 20
article already present
Serpens
article already present
Sculptor Group
article already present
State (polity)
article already present
Stevia
article already present
Search for extraterrestrial intelligence
article already present
Sextans
article already present
Salem al-Hazmi
article already present
Satsuma Province
article already present
Scottish
article already present
List of brightest stars
article already present
List of nearest stars and brown dwarf

Simplified molecular-input line-entry system
article already present
Soweto
article already present
Sampo
article already present
Sysop
article already present
Skuld (Oh My Goddess!)
article already present
Steve Jackson
article already present
Springbok
article already present
Song of Songs
article already present
Sheridan Le Fanu
article already present
Star Chamber
article already present
Coat of arms of South Africa
article already present
Shoah (disambiguation)
article already present
Second Epistle to the Corinthians
article already present
Specialization (logic)
article already present
Shaka
article already present
Sinusitis
article already present
Aos Sí
article already present
Star cluster
article already present
Scarlatti
article already present
Seinfeld
article already present
Sergei Diaghilev
article already present
Soong Mei-ling
article already present
Soong sisters
article already present
Second Epistle of Peter
article already present
Second Epistle of John
article alre

Cyanoacrylate
article already present
Shell script
article already present
Subtitle (disambiguation)
article already present
Solder
article already present
Shōgun
article already present
Short-term memory
article already present
State supreme court
article already present
Stability
article already present
Spyware
article already present
William Jones (philologist)
article already present
Stephen, King of England
article already present
Space Battleship Yamato
article already present
Southern blot
article already present
Standard-gauge railway
article already present
Sodium laureth sulfate
article already present
Saraswati River (disambiguation)
article already present
Simpson Desert
article already present
Skycar
article already present
Silesian Voivodeship
article already present
SECD machine
article already present
Stratego
article already present
Sindh
article already present
Super Bowl III
article already present
Super Bowl XX
article already present
Salute
article already present


article already present
Slope
article already present
Sex organ
article already present
Snake
article already present
Steam turbine
article already present
Sardinia
article already present
Scrooge McDuck
article already present
Shiva (Judaism)
article already present
Semi-trailer truck
article already present
Stonewall riots
article already present
Sheffer stroke
article already present
Stalactite
article already present
Strangers in Paradise
article already present
Summer
article already present
Spring
article already present
Shrike
article already present
Screwdriver (cocktail)
article already present
Single-stage-to-orbit
article already present
Structural biology
article already present
Sunni Islam
article already present
Sour mix
article already present
Sikhs
article already present
Superworld
article already present
Samuel Taylor Coleridge
article already present
Spica
article already present
Stuart Little
article already present
Statite
article already present
Stanford (disambig

article already present
The Sound of Music
article already present
Trip hop
article already present
Tycho Brahe
article already present
The A-Team
article already present
Terry Pratchett
article already present
Treaty of Versailles
article already present
Mort
article already present
Tim Berners-Lee
article already present
The Legend of Zelda
article already present
Tor Nørretranders
article already present
Thomas Henry Huxley
article already present
Triumph of the Will
article already present
Titanium
article already present
Technetium
article already present
Tin
article already present
Tellurium
article already present
Thorium
article already present
Terbium
article already present
Tungsten
article already present
Thulium
article already present
Tantalum
article already present
Torch
article already present
Thule (disambiguation)
article already present
The Qt Company
article already present
Trojan horse (computing)
article already present
Tokyo
article already present
Trojan War
art